Program 4:
Use word embeddings to improve prompts for Generative AI model. Retrieve similar words using word embeddings. Use the similar words to enrich a GenAI prompt. Use the AI model to generate responses for the original and enriched prompts. Compare the outputs in terms of detail and relevance.

In [ ]:
# Install required libraries
! pip install gensim transformers nltk matplotlib

# Import libraries
import gensim.downloader as api
from transformers import pipeline
import nltk
import string
from nltk.tokenize import word_tokenize

# Download required NLTK resources
nltk.download('punkt')  # Correct resource for tokenization
nltk.download('punkt_tab')

# Load pre-trained word vectors
print("Loading pre-trained word vectors...")
word_vectors = api.load("glove-wiki-gigaword-100")  # You can change to another model if desired

# Function to replace a keyword in the prompt with its most similar word
def replace_keyword_in_prompt(prompt, keyword, word_vectors, topn=1):
    words = word_tokenize(prompt)
    enriched_words = []
    for word in words:
        cleaned_word = word.lower().strip(string.punctuation)
        if cleaned_word == keyword.lower():
            try:
                similar_words = word_vectors.most_similar(cleaned_word, topn=topn)
                if similar_words:
                    replacement_word = similar_words[0][0]
                    print(f"Replacing '{word}' → '{replacement_word}'")
                    enriched_words.append(replacement_word)
                    continue  # Skip appending original
            except KeyError:
                print(f"'{keyword}' not found in the vocabulary. Using original word.")
        enriched_words.append(word)
    enriched_prompt = " ".join(enriched_words)
    print(f"\n🔹 Enriched Prompt: {enriched_prompt}")
    return enriched_prompt

# Load GPT-2 model for text generation
print("\nLoading GPT-2 model...")
generator = pipeline("text-generation", model="gpt2")

# Function to generate response using GPT-2
def generate_response(prompt, max_length=100):
    try:
        response = generator(prompt, max_length=max_length, num_return_sequences=1)
        return response[0]['generated_text']
    except Exception as e:
        print(f"Error generating response: {e}")
        return None

# Example original prompt
original_prompt = "Who is king."
print(f"\n🔹 Original Prompt: {original_prompt}")

# Define the keyword to be enriched
key_term = "king"

# Enrich the prompt
enriched_prompt = replace_keyword_in_prompt(original_prompt, key_term, word_vectors)

# Generate responses
print("\nGenerating response for the original prompt...")
original_response = generate_response(original_prompt)
print("\nOriginal Prompt Response:")
print(original_response)

print("\nGenerating response for the enriched prompt...")
enriched_response = generate_response(enriched_prompt)
print("\nEnriched Prompt Response:")
print(enriched_response)

# Comparison
print("\n📊 Comparison of Responses:")
print("Original Prompt Response Length:", len(original_response))
print("Enriched Prompt Response Length:", len(enriched_response))
print("Original Prompt Sentences:", original_response.count('.'))
print("Enriched Prompt Sentences:", enriched_response.count('.'))